In [1]:
import pandas as pd
import numpy as np

print("Project 3 environment ready")


Project 3 environment ready


In [2]:
data = [
    ["u001", "2025-01-01 08:01:00", "192.168.1.10", 0],
    ["u001", "2025-01-01 08:02:00", "192.168.1.10", 0],
    ["u001", "2025-01-01 08:03:00", "192.168.1.10", 0],
    ["u001", "2025-01-01 08:04:00", "192.168.1.10", 0],
    ["u001", "2025-01-01 08:05:00", "192.168.1.10", 0],

    ["u002", "2025-01-01 09:10:00", "10.0.0.5", 0],
    ["u002", "2025-01-01 09:11:00", "10.0.0.5", 1],

    ["u999", "2025-01-01 01:00:00", "203.0.113.9", 0],
    ["u999", "2025-01-01 01:01:00", "203.0.113.9", 0],
    ["u999", "2025-01-01 01:02:00", "203.0.113.9", 0],
    ["u999", "2025-01-01 01:03:00", "203.0.113.9", 0],
    ["u999", "2025-01-01 01:04:00", "203.0.113.9", 0],
]

df = pd.DataFrame(
    data,
    columns=["user_id", "timestamp", "ip", "success"]
)

df["timestamp"] = pd.to_datetime(df["timestamp"])
df


,user_id,timestamp,ip,success
0,u001,2025-01-01 08:01:00,192.168.1.10,0
1,u001,2025-01-01 08:02:00,192.168.1.10,0
2,u001,2025-01-01 08:03:00,192.168.1.10,0
3,u001,2025-01-01 08:04:00,192.168.1.10,0
4,u001,2025-01-01 08:05:00,192.168.1.10,0
5,u002,2025-01-01 09:10:00,10.0.0.5,0
6,u002,2025-01-01 09:11:00,10.0.0.5,1
7,u999,2025-01-01 01:00:00,203.0.113.9,0
8,u999,2025-01-01 01:01:00,203.0.113.9,0
9,u999,2025-01-01 01:02:00,203.0.113.9,0


In [3]:
failed = df[df["success"] == 0]
failed = failed.sort_values("timestamp")
failed



,user_id,timestamp,ip,success
7,u999,2025-01-01 01:00:00,203.0.113.9,0
8,u999,2025-01-01 01:01:00,203.0.113.9,0
9,u999,2025-01-01 01:02:00,203.0.113.9,0
10,u999,2025-01-01 01:03:00,203.0.113.9,0
11,u999,2025-01-01 01:04:00,203.0.113.9,0
0,u001,2025-01-01 08:01:00,192.168.1.10,0
1,u001,2025-01-01 08:02:00,192.168.1.10,0
2,u001,2025-01-01 08:03:00,192.168.1.10,0
3,u001,2025-01-01 08:04:00,192.168.1.10,0
4,u001,2025-01-01 08:05:00,192.168.1.10,0


In [4]:
alerts = []

for (user, ip), group in failed.groupby(["user_id", "ip"]):
    group = group.sort_values("timestamp")
    time_diff = group["timestamp"].diff().dt.total_seconds() / 60

    if (time_diff <= 5).sum() >= 4:
        alerts.append({
            "user_id": user,
            "ip": ip,
            "failed_attempts": len(group),
            "first_attempt": group["timestamp"].min(),
            "last_attempt": group["timestamp"].max()
        })

alerts_df = pd.DataFrame(alerts)
alerts_df


,user_id,ip,failed_attempts,first_attempt,last_attempt
0,u001,192.168.1.10,5,2025-01-01 08:01:00,2025-01-01 08:05:00
1,u999,203.0.113.9,5,2025-01-01 01:00:00,2025-01-01 01:04:00


In [5]:
alerts_df["risk_score"] = alerts_df["failed_attempts"] * 10
alerts_df.sort_values(by="risk_score", ascending=False)



,user_id,ip,failed_attempts,first_attempt,last_attempt,risk_score
0,u001,192.168.1.10,5,2025-01-01 08:01:00,2025-01-01 08:05:00,50
1,u999,203.0.113.9,5,2025-01-01 01:00:00,2025-01-01 01:04:00,50


## Project 3: User-Level Risk Scoring

This project extends rule-based brute-force detection by aggregating
failed login behavior at the user level and assigning an interpretable
risk score.

Rather than treating all alerts equally, this approach prioritizes
users based on the volume and temporal density of failed login attempts.
Such prioritization is critical in real-world security operations,
where analysts must triage limited investigative resources.

The risk score implemented here is intentionally simple and
interpretable, reflecting a design choice favoring transparency over
complexity. In production systems, this score could be enhanced with
additional signals such as IP reputation, geographic distance, device
fingerprinting, or historical user behavior.

This project demonstrates the transition from event-level detection to
user-centric risk assessment, a key concept in cybersecurity analytics
and applied machine learning.
